## Debate Transcripts 

Links:

[CPD Commission of Presidential Debates](https://www.debates.org/voter-education/debate-transcripts/)

In [32]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style = "fivethirtyeight"
import json
import numpy as np
from os import listdir
from os.path import isfile, join
import pymongo

import re
import itertools
from spacy.lang.en import English

In [2]:
## Transcripts files

In [47]:
filenames = list(listdir('raw_transcripts'))

In [50]:
filenames

['October 11, 1984- The Bush-Ferraro Vice Presidential Debate.txt',
 'October 11, 1992- The First Clinton-Bush-Perot Presidential Debate.txt',
 'October 11, 2000- The Second Gore-Bush Presidential Debate.txt',
 'October 11, 2012- The Biden-Ryan Vice Presidential Debate.txt',
 'October 13, 1960- The Third Kennedy-Nixon Presidential Debate.txt',
 'October 13, 1988- The Second Bush-Dukakis Presidential Debate.txt',
 'October 13, 1992- The Gore-Quayle-Stockdale Vice Presidential Debate.txt',
 'October 13, 2004- The Third Bush-Kerry Presidential Debate.txt',
 'October 15, 1992- The Second Clinton-Bush-Perot Presidential Debate.txt',
 'October 15, 2008- The Third McCain-Obama Presidential Debate.txt',
 'October 16, 1996- The Second Clinton-Dole Presidential Debate.txt',
 'October 16, 2012- The Second Obama-Romney Presidential Debate.txt',
 'October 17, 2000- The Third Gore-Bush Presidential Debate.txt',
 'October 19, 1992- The Third Clinton-Bush-Perot Presidential Debate.txt',
 'October 19, 

In [93]:
date_str = ['1984-10-11-', '1992-10-11-', '2000-10-11-', '2012-10-11-', '1960-10-13-', '1988-10-13-', 
            '1992-10-13-', '2004-10-13-', '1992-10-15-', '2008-10-15-', '1996-10-16-', '2012-10-16-',
            '2000-10-17-', '1992-10-19-', '2016-10-19-', '2008-10-02-', '1960-10-21-', '1984-10-21-',
            '1976-10-22-', '2012-10-22-', '1980-10-28-', '2000-10-03-', '2012-10-03-', '1988-10-05-',
            '2000-10-05-', '2004-10-05-', '1976-10-06-', '1996-10-06-', '1960-10-07-', '1984-10-07-', 
            '2008-10-07-', '2004-10-08-', '1996-10-09-', '2016-10-09-', '2016-10-04-', '1980-09-21-',
            '1976-09-23-', '1988-09-25-', '1960-09-26-', '2008-09-26-', '2004-09-30-', '2016-09-26-']
len(set(date_str))

42

In [49]:
filenames.pop(0)

'.DS_Store'

In [10]:
f = open(filenames[0], "r")
raw_transcript = f.read()
print(raw_transcript)

Presidential Debate at Hofstra University in Hempstead, New YorkSeptember 26, 2016PARTICIPANTS:Former Secretary of State Hillary Clinton (D) andBusinessman Donald Trump (R)MODERATOR:Lester Holt (NBC News)
HOLT: Good evening from Hofstra University in Hempstead, New York. I’m Lester Holt, anchor of “NBC Nightly News.” I want to welcome you to the first presidential debate.The participants tonight are Donald Trump and Hillary Clinton. This debate is sponsored by the Commission on Presidential Debates, a nonpartisan, nonprofit organization. The commission drafted tonight’s format, and the rules have been agreed to by the campaigns.The 90-minute debate is divided into six segments, each 15 minutes long. We’ll explore three topic areas tonight: Achieving prosperity; America’s direction; and securing America. At the start of each segment, I will ask the same lead-off question to both candidates, and they will each have up to two minutes to respond. From that point until the end of the segmen

### September 26, 2016
Presidential Debate at Hofstra University in Hempstead, New York
- Moderator(s): HOLT

Participants:

+ Democrat: CLINTON
+ Republican: TRUMP

In [11]:
# Removes [applause] and non-speaking notes
speaker_transcript = re.sub(r'\[\w+\]', '', raw_transcript)

In [21]:
# Splits the text into lines by each speaker
line_pattern = re.compile(r'(HOLT: |CLINTON: |TRUMP: )(.*)')
lines = line_pattern.finditer(speaker_transcript)
for idx, line in enumerate(lines):
    speaker=''
    if line.group(1)==('CLINTON: '):
        speaker='Democrat'
    elif line.group(1)==('TRUMP: '):
        speaker='Republican'
    elif line.group(1)==('HOLT: '):
        speaker='Moderator'
    lines_transcript.append({'line_num': idx, 'speaker': speaker, 'name': line.group(1), 'line': line.group(2)})
lines_transcript

[{'line_num': 0,
  'speaker': 'Moderator',
  'name': 'HOLT: ',
  'line': 'Good evening from Hofstra University in Hempstead, New York. I’m Lester Holt, anchor of “NBC Nightly News.” I want to welcome you to the first presidential debate.The participants tonight are Donald Trump and Hillary Clinton. This debate is sponsored by the Commission on Presidential Debates, a nonpartisan, nonprofit organization. The commission drafted tonight’s format, and the rules have been agreed to by the campaigns.The 90-minute debate is divided into six segments, each 15 minutes long. We’ll explore three topic areas tonight: Achieving prosperity; America’s direction; and securing America. At the start of each segment, I will ask the same lead-off question to both candidates, and they will each have up to two minutes to respond. From that point until the end of the segment, we’ll have an open discussion.The questions are mine and have not been shared with the commission or the campaigns. The audience here 

In [77]:
# Retrieve the speakers from each raw_transcript
speakers = []
for filename in filenames:
    path = 'raw_transcripts/'+str(filename)
    f = open(path, "r")
    speaker_pattern = re.compile(r'[A-Zc. ]{2,}: ')
    matches = speaker_pattern.finditer(f.read())
    for match in matches:
        speakers.append(match.group())
speakers = list(set(speakers))

In [78]:
# clean speakers
prefixes = ('.', ' ')
speakers = [x for x in speakers if not x.startswith(prefixes)]
speakers.pop()
speakers

['QUARLES: ',
 'SHAW: ',
 'MR. LEVY: ',
 'MR. MONDALE: ',
 'MR. CATER: ',
 'MR. GOLDFARB: ',
 'DOROTHY S. RIDINGS: ',
 'MR. CRONKITE: ',
 'AUDIENCE QUESTION: ',
 'MR. HILLIARD: ',
 'BOYD: ',
 'MR. NIXON: ',
 'LONG: ',
 'GIBSON: ',
 'GOVERNOR CARTER: ',
 'MR. BARNES: ',
 'LIEBERMAN: ',
 'MR. DRUMMOND: ',
 'MR. KRAFT: ',
 'GIBBONS: ',
 'SCHIEFFER: ',
 'VARNER: ',
 'MR. NEWMAN: ',
 'MASHEK: ',
 'MR. KITE: ',
 'KAINE: ',
 'HOLT: ',
 'PARTICIPANTS: ',
 'MR. WIEGHART: ',
 'HARRY ELLIS: ',
 'THOMAS: ',
 'MS. FREDERIC: ',
 'TRUMP: ',
 'OBAMA: ',
 'PRESIDENT GEORGE BUSH: ',
 'ROMNEY: ',
 'MAY: ',
 'GOVERNOR CLINTON: ',
 'MR. MILLIGAN: ',
 'MS. SIEFERT: ',
 'QUAYLE: ',
 'MR. SHADEL: ',
 'VOICE: ',
 'DAHLE: ',
 'MR. VANOCUR: ',
 'ROOK: ',
 'CAROLE SIMPSON: ',
 'MR. McGEE: ',
 'QUIJANO: ',
 'PENCE: ',
 'WASHINGTON POST: ',
 'REP. JOHN B. ANDERSON: ',
 'MR. MAYNARD: ',
 'BENTSEN: ',
 'MR. KENNEDY: ',
 'WARNER: ',
 'MR. TREWHITT: ',
 'MR. EDWARDS: ',
 'MR. VALERIANI: ',
 'SPEAKERS: ',
 'MS. GIANNOTT

In [79]:
# Categorize everyone!
democrats, republicans, independents, moderators = [], [], [], []
dem_candidates = ['CLINTON', 'KAINE', 'OBAMA', 'BIDEN', 'KERRY', 'EDWARDS', 'GORE', 'LIEBERMAN', 'DUKAKIS',
                  'BENTSEN', 'MONDALE', 'FERRARO', 'CARTER', 'KENNEDY', 'JOHNSON']
rep_candidates = ['TRUMP', 'PENCE', 'ROMNEY', 'RYAN', 'MCCAIN', 'PALIN', 'BUSH', 'CHENEY', 'DOLE', 'KEMP', 
                  'QUAYLE', 'REAGAN', 'FORD', 'DOLE', 'NIXON']
ind_candidates = ['PEROT', 'STOCKDALE']

for dem in dem_candidates:
    for speaker_d in speakers:
        if dem in speaker_d:
            democrats.append(speaker_d)
            speakers.remove(speaker_d)

for rep in rep_candidates:
    for speaker_r in speakers:
        if rep in speaker_r:
            republicans.append(speaker_r)
            speakers.remove(speaker_r)
            
for ind in ind_candidates:
    for speaker_i in speakers:
        if ind in speaker_i:
            independents.append(speaker_i)
            speakers.remove(speaker_i)
            
moderators = speakers
moderators.remove('THE PRESIDENT: ')
republicans.append('THE PRESIDENT: ')
moderators.remove('PARTICIPANTS: ')

['QUARLES: ',
 'SHAW: ',
 'MR. LEVY: ',
 'MR. CATER: ',
 'MR. GOLDFARB: ',
 'DOROTHY S. RIDINGS: ',
 'MR. CRONKITE: ',
 'AUDIENCE QUESTION: ',
 'MR. HILLIARD: ',
 'BOYD: ',
 'LONG: ',
 'GIBSON: ',
 'MR. BARNES: ',
 'MR. DRUMMOND: ',
 'MR. KRAFT: ',
 'GIBBONS: ',
 'SCHIEFFER: ',
 'VARNER: ',
 'MR. NEWMAN: ',
 'MASHEK: ',
 'MR. KITE: ',
 'HOLT: ',
 'PARTICIPANTS: ',
 'MR. WIEGHART: ',
 'HARRY ELLIS: ',
 'THOMAS: ',
 'MS. FREDERIC: ',
 'MAY: ',
 'MR. MILLIGAN: ',
 'MS. SIEFERT: ',
 'MR. SHADEL: ',
 'VOICE: ',
 'DAHLE: ',
 'MR. VANOCUR: ',
 'ROOK: ',
 'CAROLE SIMPSON: ',
 'MR. McGEE: ',
 'QUIJANO: ',
 'WASHINGTON POST: ',
 'REP. JOHN B. ANDERSON: ',
 'MR. MAYNARD: ',
 'WARNER: ',
 'MR. TREWHITT: ',
 'MR. VALERIANI: ',
 'SPEAKERS: ',
 'MS. GIANNOTTI: ',
 'AUDIENCE MEMBER: ',
 'MR. REYNOLDS: ',
 'MR. GANNON: ',
 'JIM LEHRER: ',
 'MR. NELSON: ',
 'MICHAELSON: ',
 'MR. KALB: ',
 'COOPER: ',
 'DEGENHART: ',
 'MR. KONDRACKE: ',
 'MS. DUBE: ',
 'SIMPSON: ',
 'MS. NAUDIN: ',
 'BRIEN: ',
 'MARGOLIS

In [84]:
dem_test_str = '('
for dem in democrats[:-1]:
    dem_test_str = dem_test_str + dem + '|'
dem_test_str = dem_test_str + democrats[-1] + ')'
dem_test_str

'(GOVERNOR CLINTON: |CLINTON: |KAINE: |OBAMA: |BIDEN: |KERRY: |MR. EDWARDS: |EDWARDS: |GORE: |LIEBERMAN: |DUKAKIS: |BENTSEN: |MR. MONDALE: |FERRARO: |GOVERNOR CARTER: |MR. CARTER: |MR. KENNEDY: |SENATOR KENNEDY: |MS. JOHNSON: )'

In [86]:
rep_test_str = '('
for rep in republicans[:-1]:
    rep_test_str = rep_test_str + rep + '|'
rep_test_str = rep_test_str + republicans[-1] + ')'
rep_test_str

'(TRUMP: |PENCE: |ROMNEY: |RYAN: |MCCAIN: |PALIN: |PRESIDENT GEORGE BUSH: |BUSH: |PRESIDENT BUSH: |CHENEY: |DOLE: |KEMP: |QUAYLE: |GOV. RONALD REAGAN: |MR. REAGAN: |REAGAN: |MR. FORD: |MR.FORD: |MR. NIXON: |THE PRESIDENT: )'

In [88]:
ind_test_str = '('
for ind in independents[:-1]:
    ind_test_str = ind_test_str + ind + '|'
ind_test_str = ind_test_str + independents[-1] + ')'
ind_test_str

'(ROSS PEROT: |PEROT: |STOCKDALE: )'

In [91]:
mod_test_str = '('
for mod in moderators[:-1]:
    mod_test_str = mod_test_str + mod + '|'
mod_test_str = mod_test_str + moderators[-1] + ')'
mod_test_str

'(QUARLES: |SHAW: |MR. LEVY: |MR. CATER: |MR. GOLDFARB: |DOROTHY S. RIDINGS: |MR. CRONKITE: |AUDIENCE QUESTION: |MR. HILLIARD: |BOYD: |LONG: |GIBSON: |MR. BARNES: |MR. DRUMMOND: |MR. KRAFT: |GIBBONS: |SCHIEFFER: |VARNER: |MR. NEWMAN: |MASHEK: |MR. KITE: |HOLT: |MR. WIEGHART: |HARRY ELLIS: |THOMAS: |MS. FREDERIC: |MAY: |MR. MILLIGAN: |MS. SIEFERT: |MR. SHADEL: |VOICE: |DAHLE: |MR. VANOCUR: |ROOK: |CAROLE SIMPSON: |MR. McGEE: |QUIJANO: |WASHINGTON POST: |REP. JOHN B. ANDERSON: |MR. MAYNARD: |WARNER: |MR. TREWHITT: |MR. VALERIANI: |SPEAKERS: |MS. GIANNOTTI: |AUDIENCE MEMBER: |MR. REYNOLDS: |MR. GANNON: |JIM LEHRER: |MR. NELSON: |MICHAELSON: |MR. KALB: |COOPER: |DEGENHART: |MR. KONDRACKE: |MS. DUBE: |SIMPSON: |MS. NAUDIN: |BRIEN: |MARGOLIS: |GROER: |BALDI: |BRUNO: |MR. STONE: |HUBB: |QUINN: |GRABEL: |WALLACE: |MR. ELLIS: |MR. NIVEN: |QUESTION: |MS. RIDINGS: |BARROW: |CONNOR: |MR. VON FREMD: |MS. STRATEGOS: |LAURENT: |MS. FREDERICK: |ANDERSON: |MS. SAWYER: |MITCHELL: |MR. HOWE: |MODERATOR: 

In [29]:
for line in lines_transcript:
    line['line_num'] = '2016-9-26-'+str(line['line_num'])

In [31]:
def clean_and_collect(index, ):
    # Removes [applause] and non-speaking notes
    speaker_transcript = re.sub(r'\[\w+\]', '', raw_transcript)
    # Splits the text into lines by each speaker
    line_pattern = re.compile(r'(HOLT: |CLINTON: |TRUMP: )(.*)')
    lines = line_pattern.finditer(speaker_transcript)
    for idx, line in enumerate(lines):
        speaker=''
        if line.group(1)==('CLINTON: '):
            speaker='Democrat'
        elif line.group(1)==('TRUMP: '):
            speaker='Republican'
        elif line.group(1)==('HOLT: '):
            speaker='Moderator'
        lines_transcript.append({'line_num': idx, 'speaker': speaker, 'name': line.group(1), 'line': line.group(2)})
    lines_transcript

'2016-9-26-57'

## MongoDB

In [ ]:
# Creates a MongoDB Database
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb = myclient['presidential_debates_db']
lab_collection = mydb['']

## Spacy NLP

In [ ]:
# Import the English language class with nlp object
nlp = English()

## Putting it all into MongoDB

In [3]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
debatedb = myclient['Debate_Topic_Modelling_db']


In [4]:
print(myclient.list_database_names())

['admin', 'config', 'local']
